<a href="https://colab.research.google.com/github/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/blob/main/Preprocessing_Notebook%20-%202nd%20edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 Defining Our Target Variables: IMDB Score and ROI

To define what makes a movie a **blockbuster**, we rely on two key target variables that capture **both audience satisfaction** and **financial performance**.

### 🟦 1. IMDB Score

The `imdb_score` columns represents the **IMDB rating** of a movie. These scores are:

- Collected from user-generated reviews on [IMDb.com](https://www.imdb.com/)
- Scored on a **scale from 1 to 10**
- Calculated using a **weighted average**, where frequent and credible users have more influence

This variable serves as a **proxy for audience reception**, making it a critical component in predicting a movie’s blockbuster potential.

<u>NOTE</u>: There's different columns that refer to this variable in different names. We'll pay attention to it and normalize everyone to one name and column in our database.

---

### 🟧 2. Return on Investment (ROI)

To capture financial success, we use **Return on Investment (ROI)**, which is a known method to measure the profit generated relative to the production cost.

By definition:

$$
\text{ROI} = \frac{\text{Gross} - \text{Budget}}{\text{Budget}}
$$

Where:
- **Budget** = How much it cost to produce the movie  
- **Gross** = Total revenue generated (worldwide or domestic, depending on dataset)

<u>NOTE</u>:: I **won’t calculate ROI just yet**, since we want to:
- First acquire all data sources
- See which tables have `gross` and `budget` fields
- Avoid recalculating ROI from incomplete or conflicting data

Once we identify the best available data, we’ll apply this formula to fill in any missing ROI values.


In [24]:
# Define a compact Column Summary Function
def quick_column_summary(df, table_name):
    print(f"\n📋 Column Summary for `{table_name}`\n")
    total_rows = len(df)
    summary = pd.DataFrame({
        'Column': df.columns,
        'Data Type': [df[col].dtype for col in df.columns],
        'NA Count': [df[col].isna().sum() for col in df.columns],
        '% Missing': [df[col].isna().mean() * 100 for col in df.columns]
    })
    display(summary)

# 1st Dataset: Movie Franchises



## Step 1: Acquisition

In [25]:
# 1. Movie Data Analysis Dataset

!wget https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/movie.csv -O movie.csv

# Load the CSV file
import pandas as pd
movie_franchises = pd.read_csv("movie.csv")

--2025-04-07 14:18:32--  https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/movie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1294548 (1.2M) [text/plain]
Saving to: ‘movie.csv’

movie.csv           100%[===================>]   1.23M  --.-KB/s    in 0.04s   

2025-04-07 14:18:32 (29.7 MB/s) - ‘movie.csv’ saved [1294548/1294548]



### First check

In [26]:
# Display the first few rows
movie_franchises.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [27]:
# Check for data types and NA's
quick_column_summary(movie_franchises, 'movie_franchises')


📋 Column Summary for `movie_franchises`



,Column,Data Type,NA Count,% Missing
0,name,object,0,0.000000
1,rating,object,77,1.004173
2,genre,object,0,0.000000
3,year,int64,0,0.000000
4,released,object,2,0.026082
5,score,float64,3,0.039124
6,votes,float64,3,0.039124
7,director,object,0,0.000000
8,writer,object,3,0.039124
9,star,object,1,0.013041


In [29]:
# Rename the IMDB score column
movie_franchises = movie_franchises.rename(columns={"score": "imdb_score"})  # replace "score" with our desirable target name - "imdb_score"

In [30]:
# Omit observations with NA's in target variables
movie_franchises = movie_franchises[
    movie_franchises['imdb_score'].notna() &
    movie_franchises['budget'].notna() &
    movie_franchises['gross'].notna()
].copy()

### 📥 Table Acquisition Summary: `movie_franchises`

#### 🎯 Relevant Variables

| Column         | Description                     | Relevance                          |
|----------------|----------------------------------|-------------------------------------|
| `name`         | Movie name (key)                | ✅ Unique ID across datasets         |
| `imdb_score`   | IMDB rating                     | ✅ Target variable #1                |
| `budget`       | Budget in dollars               | 📌 Required for ROI (target #2)     |
| `gross`        | Revenue in dollars              | 📌 Required for ROI (target #2)     |
| `votes`        | Number of user ratings          | 🧪 May influence IMDB score         |
| `genre`, `rating`, `year`, `released` | Movie metadata | 📊 Potential features |
| `director`, `writer`, `star`, `company` | People / studio involved | 📊 Potential features |
| `runtime`      | Duration in minutes             | 📊 Feature (e.g., audience fatigue) |
| `country`      | Country of production           | 📊 Feature for cultural reception   |

## Step 2: Normalize Movie Names Across Tables & Subset Columns
That's absolutely essential - When merging across tables, we’ll apply a function like that creates a unified key: `movie_id`

We’ll use this method across all tables.

Moreover, we'll create standardized tables inside the notebook
Split into:

* `imdb_score_features` (if relevant)

* `roi_features` (if relevant)

In [31]:
def normalize_movie_title(title):
    return title.strip().lower()

In [32]:
# Normalize movie name for joining
movie_franchises['movie_id'] = movie_franchises['name'].str.strip().str.lower()

# Subset score-related features
imdb_score_features_1 = movie_franchises[[
    'movie_id', 'imdb_score', 'votes', 'genre', 'rating', 'year', 'runtime', 'country',
    'director', 'writer', 'star', 'company'
]]

# Subset ROI-related features
roi_features_1 = movie_franchises[['movie_id', 'gross', 'budget']]

# 2nd Dataset: additional Movie Franchises

## Step 1: Acquisition

In [33]:
# 2. Global Movie Franchise Revenue and Budget Data

!wget https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/MovieFranchises.csv -O MovieFranchises.csv
import pandas as pd
data2 = pd.read_csv("MovieFranchises.csv") # Save in a different name due to similar name to the 1st dataset

--2025-04-07 14:19:04--  https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/MovieFranchises.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26322 (26K) [text/plain]
Saving to: ‘MovieFranchises.csv’

MovieFranchises.csv 100%[===================>]  25.71K  --.-KB/s    in 0.002s  

2025-04-07 14:19:04 (10.9 MB/s) - ‘MovieFranchises.csv’ saved [26322/26322]



### First check

In [34]:
# Display the first few rows
data2.head()

,index,MovieID,Title,Lifetime Gross,Year,Studio,Rating,Runtime,Budget,ReleaseDate,VoteAvg,VoteCount,FranchiseID
0,0,1001,Star Wars: Episode IV - A New Hope,775398007,1977,Lucasfilm,PG,121.0,11000000.0,05-25-77,4.09,96233.0,101.0
1,1,1002,Star Wars: Episode V - The Empire Strikes Back,538375067,1980,Lucasfilm,PG,124.0,18000000.0,06-20-80,4.12,79231.0,101.0
2,2,1003,Star Wars: Episode VI - Return of the Jedi,475106177,1983,Lucasfilm,PG,135.0,32500000.0,05-25-83,3.98,76082.0,101.0
3,3,1004,Jurassic Park,1109802321,1993,Universal Pictures,PG-13,127.0,63000000.0,06-11-93,3.69,82700.0,102.0
4,4,1005,The Lost World: Jurassic Park,618638999,1997,Universal Pictures,PG-13,129.0,73000000.0,05-23-97,3.01,19721.0,102.0


In [35]:
# Check for data types and NA's
quick_column_summary(data2, 'data2')


📋 Column Summary for `data2`



,Column,Data Type,NA Count,% Missing
0,index,int64,0,0.000000
1,MovieID,object,0,0.000000
2,Title,object,0,0.000000
3,Lifetime Gross,object,0,0.000000
4,Year,object,539,89.090909
5,Studio,object,545,90.082645
6,Rating,object,545,90.082645
7,Runtime,float64,545,90.082645
8,Budget,float64,545,90.082645
9,ReleaseDate,object,545,90.082645


In [36]:
# Keep only the useful parts of data2
data2 = data2[['MovieID', 'Title', 'Lifetime Gross']].copy()

### 📥 Table Acquisition Summary: `data2`

This table had lots of missing values. But still, the table includes financial data that can add us more information about our target variable ROI.

#### 🔁 Remaining Variables

| `movie_franchises` | `studio_financials` | Action |
|--------------------|---------------------|--------|
| `name`             | `Title`             | Normalize to `movie_id` for matching |
| `budget`           | `Budget`            | Compare and retain best version |
| `gross`            | `Lifetime Gross`    | Compare with `gross` |


## Step 2: Normalize Movie Names Across Tables & Subset Columns
As we said, we’ll apply a function that creates a unified key: `movie_id`. Moreover, we'll subset the columns based on the summary

In [38]:
# Normalize movie name for joining
data2['movie_id'] = data2['Title'].str.strip().str.lower()

# Subset ROI-related features
roi_features_2 = data2[[
    'movie_id', 'Lifetime Gross'
]]

# 3rd Dataset: TMDB data

## Step 1: Acquisition

In [39]:
# 3. TMDB 5000 Movies Dataset

!pip install datasets

from datasets import load_dataset
import pandas as pd

# Load the TMDB dataset from Hugging Face
dataset = load_dataset("AiresPucrs/tmdb-5000-movies", split="train")
tmdb_data = pd.DataFrame(dataset)

# Save the DataFrame to a CSV file
tmdb_data.to_csv("tmdb_movies.csv", index=False)

# Confirm the file exists in the current directory
import os
os.listdir()

['.config',
 'movie.csv',
 'tmdb_movies.csv',
 'MovieFranchises.csv',
 'sample_data']

### First Check

In [41]:
# Display the first few rows
tmdb_data.head()

,id,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,5,4000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...",None,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...",en,Four Rooms,It's Ted the Bellhop's first night on the job....,22.876230,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,4300000,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,530,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,11000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",http://www.starwars.com/films/star-wars-episod...,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...",en,Star Wars,Princess Leia is captured and held hostage by ...,126.393695,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...",...,775398007,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6624,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,94000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://movies.disney.com/finding-nemo,"[{""id"": 494, ""name"": ""father son relationship""...",en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",85.688789,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,940335536,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,6122,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",None,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...",en,Forrest Gump,A man with a low IQ has accomplished great thi...,138.133331,"[{""name"": ""Paramount Pictures"", ""id"": 4}]",...,677945399,142.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7927,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,15000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.dreamworks.com/ab/,"[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...",en,American Beauty,"Lester Burnham, a depressed suburban father in...",80.878605,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...",...,356296601,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Look closer.,American Beauty,7.9,3313,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."


In [40]:
# Check for data types and NA's
quick_column_summary(tmdb_data, 'tmdb_data')


📋 Column Summary for `tmdb_data`



,Column,Data Type,NA Count,% Missing
0,id,int64,0,0.000000
1,budget,int64,0,0.000000
2,genres,object,0,0.000000
3,homepage,object,3091,64.355611
4,keywords,object,0,0.000000
5,original_language,object,0,0.000000
6,original_title,object,0,0.000000
7,overview,object,3,0.062461
8,popularity,float64,0,0.000000
9,production_companies,object,0,0.000000


### 📥 Table Acquisition: `tmdb_data`

This is the richest and most structured table so far. It includes both structured and nested (JSON-like) data, contributing heavily to both prediction targets.

---

#### 🎯 Relevant Variables

| Column | Description | Relevance |
|--------|-------------|-----------|
| `title` | Movie name | ✅ Used to create `movie_id` |
| `vote_average` | Average audience rating | ✅ Proxy for IMDB score |
| `vote_count` | Number of votes | 🧪 May influence or complement score |
| `budget` | Production cost | 📌 Required for ROI |
| `revenue` | Box office revenue | 📌 Required for ROI |
| `runtime` | Duration in minutes | 📊 Feature for pacing / cost |
| `popularity` | TMDB popularity score | 📊 Social visibility |
| `release_date` | Date released | 📊 Use for time features (month, year) |
| `genres` | List of genres (JSON) | 🧠 To parse later for genre-based analysis |
| `keywords` | Thematic keywords (JSON) | 🧠 Useful after parsing |
| `overview`, `tagline` | Textual summary & tagline | 🧠 Potential for NLP sentiment modeling |
| `original_language` | Language code (e.g., 'en') | 📊 Cultural/demographic indicator |
| `production_companies` | Companies involved (JSON) | 🧠 Feature engineering (studio power) |
| `production_countries` | Countries involved (JSON) | 📊 International impact |
| `spoken_languages` | Languages spoken (JSON) | 📊 Audience reach |
| `cast`, `crew` | Cast and crew (JSON) | 🧠 Feature-rich, parse later |
| `status` | e.g., Released, Post-production, etc. | 🧪 May correlate with box office |

---

#### 🧠 Summary

- This table contributes to both `imdb_score_features` and `roi_features`
- Contains multiple nested fields that will be parsed during feature engineering
- Will be save in SQLite as `raw_tmdb_data`


## Step 2: Normalize Movie Names Across Tables & Subset Columns
As we said, we’ll apply a function that creates a unified key: `movie_id`. Moreover, we'll subset the columns based on the summary

In [42]:
# Normalize movie name for joining
tmdb_data['movie_id'] = tmdb_data['title'].str.strip().str.lower()

# Subset score-related features
imdb_score_features_3 = tmdb_data[[
    'movie_id', 'vote_average', 'vote_count', 'popularity',
    'genres', 'keywords', 'original_language',
    'overview', 'tagline', 'status'
]]

# Subset ROI-related features
roi_features_3 = tmdb_data[[
    'movie_id', 'budget', 'revenue', 'runtime', 'release_date',
    'production_companies', 'production_countries', 'spoken_languages'
]]

# 4th Dataset: Meta-Analysis Data

## Step 1: Acquisition

In [43]:
# 4. Complete Movie Metadata Dataset

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
file_path = '/content/drive/My Drive/Projects/Blockbuster Movies/movies.csv'  # Adjust path as needed
meta_data = pd.read_csv(file_path)

# Save the DataFrame to a CSV file
meta_data.to_csv("movies.csv", index=False)

# Confirm the file exists in the current directory
import os
os.listdir()

Mounted at /content/drive


['.config',
 'movie.csv',
 'tmdb_movies.csv',
 'MovieFranchises.csv',
 'drive',
 'movies.csv',
 'sample_data']

### First Check

In [44]:
# Display the first few rows
meta_data.head()

,id,title,genres,original_language,overview,popularity,production_companies,release_date,budget,revenue,runtime,status,tagline,vote_average,vote_count,credits,keywords,poster_path,backdrop_path,recommendations
0,615656,Meg 2: The Trench,Action-Science Fiction-Horror,en,An exploratory dive into the deepest depths of...,8763.998,Apelles Entertainment-Warner Bros. Pictures-di...,2023-08-02,129000000.0,3.520565e+08,116.0,Released,Back for seconds.,7.079,1365.0,Jason Statham-Wu Jing-Shuya Sophia Cai-Sergio ...,based on novel or book-sequel-kaiju,/4m1Au3YkjqsxF8iwQy0fPYSxE0h.jpg,/qlxy8yo5bcgUw2KAmmojUKp4rHd.jpg,1006462-298618-569094-1061181-346698-1076487-6...
1,758323,The Pope's Exorcist,Horror-Mystery-Thriller,en,Father Gabriele Amorth Chief Exorcist of the V...,5953.227,Screen Gems-2.0 Entertainment-Jesus & Mary-Wor...,2023-04-05,18000000.0,6.567582e+07,103.0,Released,Inspired by the actual files of Father Gabriel...,7.433,545.0,Russell Crowe-Daniel Zovatto-Alex Essoe-Franco...,spain-rome italy-vatican-pope-pig-possession-c...,/9JBEPLTPSm0d1mbEcLxULjJq9Eh.jpg,/hiHGRbyTcbZoLsYYkO4QiCLYe34.jpg,713704-296271-502356-1076605-1084225-1008005-9...
2,533535,Deadpool & Wolverine,Action-Comedy-Science Fiction,en,A listless Wade Wilson toils away in civilian ...,5410.496,Marvel Studios-Maximum Effort-21 Laps Entertai...,2024-07-24,200000000.0,1.326387e+09,128.0,Released,Come together.,7.765,3749.0,Ryan Reynolds-Hugh Jackman-Emma Corrin-Matthew...,hero-superhero-anti hero-mutant-breaking the f...,/8cdWjvZQUExUUTzyp4t6EDMubfO.jpg,/dvBCdCohwWbsP5qAaglOXagDMtk.jpg,573435-519182-957452-1022789-945961-718821-103...
3,667538,Transformers: Rise of the Beasts,Action-Adventure-Science Fiction,en,When a new threat capable of destroying the en...,5409.104,Skydance-Paramount-di Bonaventura Pictures-Bay...,2023-06-06,200000000.0,4.070455e+08,127.0,Released,Unite or fall.,7.340,1007.0,Anthony Ramos-Dominique Fishback-Luna Lauren V...,peru-alien-end of the world-based on cartoon-b...,/gPbM0MK8CP8A174rmUwGsADNYKD.jpg,/woJbg7ZqidhpvqFGGMRhWQNoxwa.jpg,496450-569094-298618-385687-877100-598331-4628...
4,693134,Dune: Part Two,Science Fiction-Adventure,en,Follow the mythic journey of Paul Atreides as ...,4742.163,Legendary Pictures,2024-02-27,190000000.0,6.838137e+08,167.0,Released,Long live the fighters.,8.300,2770.0,Timothée Chalamet-Zendaya-Rebecca Ferguson-Jav...,epic-based on novel or book-fight-sandstorm-sa...,/czembW0Rk1Ke7lCJGahbOhdCuhV.jpg,/xOMo8BRK7PfcJv9JCnx7s5hj0PX.jpg,438631-763215-792307-1011985-467244-634492-359...


In [45]:
# Check for data types and NA's
quick_column_summary(meta_data, 'meta_data')


📋 Column Summary for `meta_data`



,Column,Data Type,NA Count,% Missing
0,id,int64,0,0.000000
1,title,object,6,0.000831
2,genres,object,210317,29.116994
3,original_language,object,0,0.000000
4,overview,object,118243,16.369959
5,popularity,float64,0,0.000000
6,production_companies,object,384926,53.290453
7,release_date,object,51549,7.136617
8,budget,float64,0,0.000000
9,revenue,float64,0,0.000000


### 📥 Table Acquisition: `meta_data`

This dataset appears to be an updated or complementary version of `tmdb_data`, containing recent and upcoming titles with similar structure.

---

#### 🎯 Relevant Variables

| Column | Description | Relevance |
|--------|-------------|-----------|
| `title` | Movie title | ✅ Used to create `movie_id` |
| `vote_average` / `vote_count` | User rating and count | ✅ Score-related |
| `budget`, `revenue` | Financial data | 📌 Used for ROI |
| `runtime`, `release_date` | Timing & length | 📊 Influences score & ROI |
| `popularity` | TMDB popularity score | 📊 Social reach |
| `genres`, `keywords`, `overview`, `tagline` | Text / tags | 🧠 Feature-rich, parse later |
| `original_language` | Language code | 📊 Cultural signal |
| `status` | Release status | 🧪 Could correlate with results |
| `production_companies` | Studios involved | 🧠 To group studio trends |
| `credits` | Raw cast and crew | 🧠 To parse later for influence modeling |

---

#### 🧠 Summary

- High overlap with `tmdb_data` (Table 3) — strong candidate for integration
- Contributes to both `imdb_score_features` and `roi_features`
- Will require deduplication and possible enrichment during post-acquisition phase
- Will be save in SQLite as `meta_data`


## Step 2: Normalize Movie Names Across Tables & Subset Columns
As we said, we’ll apply a function that creates a unified key: `movie_id`. Moreover, we'll subset the columns based on the summary

In [46]:
# Normalize movie name for joining
meta_data['movie_id'] = meta_data['title'].str.strip().str.lower()

# Subset score-related features
imdb_score_features_4 = meta_data[[
    'movie_id', 'vote_average', 'vote_count', 'popularity',
    'genres', 'keywords', 'overview', 'original_language',
    'tagline', 'status', 'credits'
]]

# Subset ROI-related features
roi_features_4 = meta_data[[
    'movie_id', 'budget', 'revenue', 'runtime', 'release_date',
    'production_companies'
]]

# 5th Dataset: Revenues Data

## Step 1: Acquisition

In [47]:
# 5. Movie Revenue Analysis Dataset

!wget https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/final_dataset.csv -O final_dataset.csv
import pandas as pd
financial_data = pd.read_csv("final_dataset.csv")

--2025-04-07 14:21:14--  https://raw.githubusercontent.com/JohnnySolo/Data-Analysis-Project---Blockbuster-Movies/main/final_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456039 (445K) [text/plain]
Saving to: ‘final_dataset.csv’

final_dataset.csv   100%[===================>] 445.35K  --.-KB/s    in 0.03s   

2025-04-07 14:21:14 (12.6 MB/s) - ‘final_dataset.csv’ saved [456039/456039]



### First Check

In [48]:
# Display the first few rows
financial_data.head()

,Unnamed: 0,movie,year,production_budget,domestic_gross,foreign_gross,worldwide_gross,month,profit,profit_margin,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,0,Avatar,2009,425000000,760507625,2015837654,2776345279,12,2351345279,0.846921,...,0,0,0,0,0,1,0,0,0,0
1,1,Pirates of the Caribbean: On Stranger Tides,2011,410600000,241063875,804600000,1045663875,5,635063875,0.607331,...,0,0,0,0,0,0,0,0,0,0
2,2,Avengers: Age of Ultron,2015,330600000,459005868,944008095,1403013963,5,1072413963,0.764364,...,0,0,0,0,0,1,0,0,0,0
3,3,Avengers: Infinity War,2018,300000000,678815482,1369318718,2048134200,4,1748134200,0.853525,...,0,0,0,0,0,0,0,0,0,0
4,4,Justice League,2017,300000000,229024295,426920914,655945209,11,355945209,0.542645,...,0,0,0,0,0,1,0,0,0,0


In [49]:
# Check for data types and NA's
quick_column_summary(financial_data, 'financial_data')


📋 Column Summary for `financial_data`



,Column,Data Type,NA Count,% Missing
0,Unnamed: 0,int64,0,0.000000
1,movie,object,0,0.000000
2,year,int64,0,0.000000
3,production_budget,int64,0,0.000000
4,domestic_gross,int64,0,0.000000
5,foreign_gross,int64,0,0.000000
6,worldwide_gross,int64,0,0.000000
7,month,int64,0,0.000000
8,profit,int64,0,0.000000
9,profit_margin,float64,0,0.000000


### 📥 Table Acquisition: `financial_data`

This table is highly focused on financial metrics and genre distribution. It provides engineered columns for ROI, profit, and genre flags, making it very valuable for prediction.

---

#### 🎯 Relevant Variables

| Column | Description | Relevance |
|--------|-------------|-----------|
| `movie` | Movie name | ✅ Used to create `movie_id` |
| `production_budget`, `domestic_gross`, `foreign_gross`, `worldwide_gross` | Raw inputs for ROI | ✅ |
| `profit`, `roi`, `profit_margin`, `pct_foreign` | Pre-calculated finance metrics | ✅ |
| `vote_average`, `vote_count`, `popularity` | Score-related audience signals | ✅ |
| `original_language`, `release_date`, `month` | Contextual/cultural features | ✅ |
| `Action`, `Drama`, etc. | Binary genre flags | ✅ Helps both score and ROI models |

---

#### 🧠 Summary

- Strongest financial data table (calculated ROI & profit)
- Includes one-hot encoded genre info (clean and ready)
- Will contribute to both `imdb_score_features` and `roi_features`
- Saved in SQLite as `raw_financial_data`


## Step 2: Normalize Movie Names Across Tables & Subset Columns
As we said, we’ll apply a function that creates a unified key: `movie_id`. Moreover, we'll subset the columns based on the summary

In [50]:
# Normalize movie name for joining
financial_data['movie_id'] = financial_data['movie'].str.strip().str.lower()

# Subset score-related features
imdb_score_features_5 = financial_data[[
    'movie_id', 'vote_average', 'vote_count', 'popularity',
    'original_language', 'release_date'
] + [g for g in financial_data.columns if g in [
    'Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 'Documentary',
    'Drama', 'Family', 'Fantasy', 'History', 'Horror', 'Music',
    'Mystery', 'Romance', 'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']]]

# Subset ROI-related features
roi_features_5 = financial_data[[
    'movie_id', 'production_budget', 'domestic_gross', 'foreign_gross',
    'worldwide_gross', 'profit', 'roi', 'profit_margin', 'pct_foreign', 'month'
]]

# Full Preprocessing & Integration Pipeline

## 🔗 Dataset Consolidation and Structure

After we finished cleaning and filtering each raw dataset, we merged them into a single unified structure using `movie_id` as the common key.

### ⚙️ Merging Logic

We performed an **outer join across all five tables** (`movie_franchises`, `data2`, `tmdb_data`, `meta_data`, `financial_data`) to make sure no potential information was lost.  
While this can create many missing values, we had already filtered each dataset in advance, so only clean and relevant observations were included in the merge.

This approach ensures we retain **every available movie** and don’t lose any potentially valuable information.

Because multiple datasets could contain the same `movie_id`, and possibly duplicated or complementary variables, we applied the following consolidation:


```python
merged.groupby('movie_id', as_index=False).first()
```

💡 Why This Works:

* For each duplicated movie_id, it keeps only one row.

* It fills each column with the first non-null value available.

* This provides a simple and effective way to consolidate information from multiple sources without manually resolving conflicts for every field.

* It’s much more efficient than complex aggregation logic like .agg(lambda x: x.dropna().iloc[0]) — especially at scale.

* The result is a clean master_movies table, where each movie is represented by a single row and all available information has been merged across sources.


In [ ]:
# 1. Define final movie_id list from the targets you trust
final_ids = set(score_features['movie_id']) | set(financial_features['movie_id'])

# 2. Filter each raw dataset
filtered_franchises = movie_franchises[movie_franchises['movie_id'].isin(final_ids)].copy()
filtered_tmdb = tmdb_data[tmdb_data['movie_id'].isin(final_ids)].copy()
filtered_meta = meta_data[meta_data['movie_id'].isin(final_ids)].copy()
filtered_data2 = data2[data2['movie_id'].isin(final_ids)].copy()
filtered_financials = financial_data[financial_data['movie_id'].isin(final_ids)].copy()

# 3. Now do an outer merge on just the relevant subset
merged = filtered_franchises \
    .merge(filtered_data2, on='movie_id', how='outer', suffixes=('', '_data2')) \
    .merge(filtered_tmdb, on='movie_id', how='outer', suffixes=('', '_tmdb')) \
    .merge(filtered_meta, on='movie_id', how='outer', suffixes=('', '_meta')) \
    .merge(filtered_financials, on='movie_id', how='outer', suffixes=('', '_finance'))

In [ ]:
#Post-merge consolidation
master_movies = merged.groupby('movie_id', as_index=False).first()

🧱 Final Dataset Structure

We then split the consolidated dataset into four logically separated, task-focused tables:

movie_master – Core identifiers for each movie, such as:

`movie_id`, `name`, `genre`, `year`

This table functions like a dimension or master key table.

score_features – Variables relevant for predicting audience satisfaction:

`imdb_score`, `votes`, `rating`, etc.

financial_features – Variables used to model financial performance and profitability:

`budget`, `gross`, `roi`, `profit`, etc.

enrichment_features – Extra metadata useful for optional or advanced modeling:

`cast`, `crew`, `homepage`, `keywords`, `tagline`, etc.

In [ ]:
# movie_master
movie_master = master_movies[[
    'movie_id', 'name', 'genre', 'year',
    'release_date', 'original_language', 'production_companies'
]].drop_duplicates()

# Check for data types and NA's
quick_column_summary(movie_master, 'movie_master')

movie_master


📋 Column Summary for `movie_master`



,Column,Data Type,NA Count,% Missing
0,movie_id,object,0,0.000000
1,name,object,0,0.000000
2,genre,object,0,0.000000
3,year,int64,0,0.000000
4,release_date,object,2235,41.767894
5,original_language,object,2235,41.767894
6,production_companies,object,2235,41.767894


,movie_id,name,genre,year,release_date,original_language,production_companies
0,*batteries not included,*batteries not included,Comedy,1987,None,None,None
1,10 cloverfield lane,10 Cloverfield Lane,Action,2016,2016-03-10,en,"[{""name"": ""Paramount Pictures"", ""id"": 4}, {""na..."
2,10 things i hate about you,10 Things I Hate About You,Comedy,1999,1999-03-30,en,"[{""name"": ""Mad Chance"", ""id"": 1757}, {""name"": ..."
3,10 to midnight,10 to Midnight,Crime,1983,None,None,None
4,"10,000 bc","10,000 BC",Action,2008,None,None,None
...,...,...,...,...,...,...,...
5346,zoolander 2,Zoolander 2,Action,2016,2016-02-06,en,"[{""name"": ""Scott Rudin Productions"", ""id"": 258..."
5347,zoom,Zoom,Action,2006,2006-08-11,en,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam..."
5348,zoot suit,Zoot Suit,Drama,1981,None,None,None
5349,zootopia,Zootopia,Animation,2016,None,None,None


In [ ]:
# score_features
score_features = master_movies[[
    'movie_id', 'imdb_score', 'vote_average', 'vote_count',
    'rating', 'runtime', 'popularity'
]].copy()

# Check for data types and NA's
quick_column_summary(score_features, 'score_features')

score_features


📋 Column Summary for `score_features`



,Column,Data Type,NA Count,% Missing
0,movie_id,object,0,0.000000
1,imdb_score,float64,0,0.000000
2,vote_average,float64,2235,41.767894
3,vote_count,float64,2235,41.767894
4,rating,object,12,0.224257
5,runtime,float64,1,0.018688
6,popularity,float64,2235,41.767894


,movie_id,imdb_score,vote_average,vote_count,rating,runtime,popularity
0,*batteries not included,6.7,NaN,NaN,PG,106.0,NaN
1,10 cloverfield lane,7.2,6.8,2468.0,PG-13,103.0,53.698683
2,10 things i hate about you,7.3,7.3,1701.0,PG-13,97.0,54.550275
3,10 to midnight,6.3,NaN,NaN,R,101.0,NaN
4,"10,000 bc",5.1,NaN,NaN,PG-13,109.0,NaN
...,...,...,...,...,...,...,...
5346,zoolander 2,4.7,4.7,797.0,PG-13,101.0,37.253774
5347,zoom,4.4,4.9,135.0,PG,83.0,10.304172
5348,zoot suit,6.8,NaN,NaN,R,103.0,NaN
5349,zootopia,8.0,NaN,NaN,PG,108.0,NaN


In [ ]:
# financial_features
financial_features = master_movies[[
    'movie_id', 'budget', 'production_budget', 'gross', 'revenue',
    'Lifetime Gross', 'worldwide_gross', 'domestic_gross',
    'profit', 'roi', 'profit_margin', 'pct_foreign'
]].copy()
financial_features['roi'] = (
    (financial_features['gross'] - financial_features['budget']) / financial_features['budget']
)

# Check for data types and NA's
quick_column_summary(financial_features, 'financial_features')

financial_features


📋 Column Summary for `financial_features`



,Column,Data Type,NA Count,% Missing
0,movie_id,object,0,0.000000
1,budget,float64,0,0.000000
2,production_budget,float64,4202,78.527378
3,gross,float64,0,0.000000
4,revenue,float64,2235,41.767894
5,Lifetime Gross,object,5299,99.028219
6,worldwide_gross,float64,4202,78.527378
7,domestic_gross,float64,4202,78.527378
8,profit,float64,4202,78.527378
9,roi,float64,0,0.000000


,movie_id,budget,production_budget,gross,revenue,Lifetime Gross,worldwide_gross,domestic_gross,profit,roi,profit_margin,pct_foreign
0,*batteries not included,25000000.0,NaN,6.508880e+07,NaN,None,NaN,NaN,NaN,1.603552,NaN,NaN
1,10 cloverfield lane,15000000.0,5000000.0,1.102170e+08,108286421.0,None,1.082864e+08,72082999.0,103286422.0,6.347800,0.953826,0.334330
2,10 things i hate about you,30000000.0,NaN,5.347858e+07,53478166.0,None,NaN,NaN,NaN,0.782619,NaN,NaN
3,10 to midnight,4520000.0,NaN,7.175592e+06,NaN,None,NaN,NaN,NaN,0.587520,NaN,NaN
4,"10,000 bc",105000000.0,NaN,2.697842e+08,NaN,None,NaN,NaN,NaN,1.569373,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5346,zoolander 2,50000000.0,50000000.0,5.672269e+07,55969000.0,None,5.534869e+07,28848693.0,5348693.0,0.134454,0.096636,0.478783
5347,zoom,35000000.0,NaN,1.250636e+07,12506188.0,None,NaN,NaN,NaN,-0.642675,NaN,NaN
5348,zoot suit,2700000.0,NaN,3.256082e+06,NaN,None,NaN,NaN,NaN,0.205956,NaN,NaN
5349,zootopia,150000000.0,150000000.0,1.024121e+09,NaN,None,1.019430e+09,341268248.0,869429616.0,5.827474,0.852859,0.665236


In [ ]:
# enrichment_features
enrichment_features = master_movies[[
    'movie_id', 'cast', 'crew', 'keywords', 'tagline',
    'overview', 'homepage', 'spoken_languages'
]].copy()

# Check for data types and NA's
quick_column_summary(enrichment_features, 'enrichment_features')

enrichment_features


📋 Column Summary for `enrichment_features`



,Column,Data Type,NA Count,% Missing
0,movie_id,object,0,0.000000
1,cast,object,2235,41.767894
2,crew,object,2235,41.767894
3,keywords,object,2235,41.767894
4,tagline,object,2486,46.458606
5,overview,object,2235,41.767894
6,homepage,object,4150,77.555597
7,spoken_languages,object,2235,41.767894


,movie_id,cast,crew,keywords,tagline,overview,homepage,spoken_languages
0,*batteries not included,None,None,None,None,None,None,None
1,10 cloverfield lane,"[{""cast_id"": 2, ""character"": ""Michelle"", ""cred...","[{""credit_id"": ""57627624c3a3680682000872"", ""de...","[{""id"": 1930, ""name"": ""kidnapping""}, {""id"": 23...",Monsters come in many forms.,"After a car accident, Michelle awakens to find...",http://www.10cloverfieldlane.com/,"[{""iso_639_1"": ""en"", ""name"": ""English""}]"
2,10 things i hate about you,"[{""cast_id"": 2, ""character"": ""Patrick Verona"",...","[{""credit_id"": ""52fe43e6c3a36847f807731d"", ""de...","[{""id"": 497, ""name"": ""shakespeare""}, {""id"": 59...",How do I loathe thee? Let me count the ways.,"Bianca, a tenth grader, has never gone on a da...",None,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso..."
3,10 to midnight,None,None,None,None,None,None,None
4,"10,000 bc",None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
5346,zoolander 2,"[{""cast_id"": 0, ""character"": ""Derek Zoolander""...","[{""credit_id"": ""54ff0d5e9251410e56001a13"", ""de...","[{""id"": 6241, ""name"": ""stupidity""}, {""id"": 966...",Long time no Z,Derek and Hansel are modelling again when an o...,http://www.zoolander.com/,"[{""iso_639_1"": ""en"", ""name"": ""English""}]"
5347,zoom,"[{""cast_id"": 1, ""character"": ""Jack Sheppard"", ...","[{""credit_id"": ""573133109251414868000f72"", ""de...","[{""id"": 9715, ""name"": ""superhero""}, {""id"": 996...",They're going to save the world... as long as ...,Jack Shepard is an out-of-shape auto shop owne...,None,"[{""iso_639_1"": ""en"", ""name"": ""English""}]"
5348,zoot suit,None,None,None,None,None,None,None
5349,zootopia,None,None,None,None,None,None,None


🧼 Missing Value Handling

Before we created these tables:

We analyzed each individual dataset using `quick_column_summary()`

We removed observations with missing values in critical variables such as:

`imdb_score` (for score modeling)

`budget` and `gross` (for ROI modeling)

Because this NA filtering was applied before the merge, and we split variables after a clean consolidation, each of the four final tables contains:

* No missing values in target-critical variables

* Columns filtered by their relevance to modeling tasks

In [ ]:
def check_complete_rows(df, table_name):
    total = len(df)
    complete = df.dropna().shape[0]
    percent_complete = (complete / total) * 100
    print(f"📊 `{table_name}`: {complete} / {total} rows complete ({percent_complete:.2f}%)")

In [ ]:
check_complete_rows(movie_master, "movie_master")
check_complete_rows(score_features, "score_features")
check_complete_rows(financial_features, "financial_features")
check_complete_rows(enrichment_features, "enrichment_features")

📊 `movie_master`: 3186 / 5438 rows complete (58.59%)
📊 `score_features`: 9992 / 17310 rows complete (57.72%)
📊 `financial_features`: 23 / 17310 rows complete (0.13%)
📊 `enrichment_features`: 3925 / 17310 rows complete (22.67%)


---